### Programa Principal:

In [1]:
from src import OWID
from utilities import DataFrameTransform
from utilities import Currency
from utilities import DataBaseSQLite3
from utilities import Type
from utilities import DataPlot
import pandas as pd
import matplotlib.pyplot as plt

### 1) Solar photovoltaic capacity:

`Extração de Dados da API OWID`

In [ ]:
solar_capacity: OWID = OWID('solar-pv-cumulative-capacity')
pdf_solar_capacity: pd.DataFrame = solar_capacity.slug_into_dataframe()
pdf_solar_capacity

`Tratamento dos nomes das Colunas`

In [ ]:
pdf_solar_capacity: pd.DataFrame = DataFrameTransform.transform_column(pdf_solar_capacity, 'entities', 'País')
pdf_solar_capacity: pd.DataFrame = DataFrameTransform.transform_column(pdf_solar_capacity, 'years', 'Ano')
pdf_solar_capacity: pd.DataFrame = DataFrameTransform.transform_column(pdf_solar_capacity, 'solar_pv_cumulative_capacity', 'Capacidade Acumulada [GW]')
pdf_solar_capacity

`Tratamento de Null`

In [ ]:
DataFrameTransform.verify_null(pdf_solar_capacity)  #Todas as linhas estão preenchidas. Sem Necessidade de tratamento!

`Tratamento tipos de dados das colunas`

In [ ]:
pdf_solar_capacity = DataFrameTransform.transform_type_column(pdf_solar_capacity, 'País', Type.STR) 
pdf_solar_capacity = DataFrameTransform.transform_type_column(pdf_solar_capacity, 'Ano', Type.INT)
pdf_solar_capacity = DataFrameTransform.transform_type_column(pdf_solar_capacity, 'Capacidade Acumulada [GW]', Type.FLOAT)

In [ ]:
pdf_solar_capacity.info()

`Agrupamento por ano no Mundo (todos os países)`

In [ ]:
solar_capacity_group_by_year: pd.DataFrame = pdf_solar_capacity.groupby(by=['Ano']).sum()
solar_capacity_group_by_year: pd.DataFrame = solar_capacity_group_by_year.drop(columns='País')
solar_capacity_group_by_year

### 2) Solar photovoltaic prices:

`Extração de Dados da API OWID`

In [ ]:
solar_prices: OWID = OWID('solar-pv-prices')
pdf_solar_prices: pd.DataFrame = solar_prices.slug_into_dataframe()
pdf_solar_prices

`Conversão de Dólar para Real`

In [ ]:
pdf_solar_prices['solar_pv_prices'] = pdf_solar_prices['solar_pv_prices'].map(lambda value: Currency(value).converter())
pdf_solar_prices

`Tratamento dos nomes das Colunas`

In [ ]:
pdf_solar_prices: pd.DataFrame = pdf_solar_prices.drop(columns='entities') #Todos são relacionadas ao Mundo todo
pdf_solar_prices: pd.DataFrame = DataFrameTransform.transform_column(pdf_solar_prices, 'years', 'Ano')
pdf_solar_prices: pd.DataFrame = DataFrameTransform.transform_column(pdf_solar_prices, 'solar_pv_prices', 'R$/W')
pdf_solar_prices

`Tratamento de Null`

In [ ]:
DataFrameTransform.verify_null(pdf_solar_prices)   #Todas as linhas estão preenchidas. Sem Necessidade de tratamento!

`Tratamento Tipo de Dados`

In [ ]:
pdf_solar_prices = DataFrameTransform.transform_type_column(pdf_solar_prices, 'Ano', Type.INT)
pdf_solar_prices = DataFrameTransform.transform_type_column(pdf_solar_prices, 'R$/W', Type.FLOAT)

In [ ]:
pdf_solar_prices.info()

### 3) Merge Dataframes

In [ ]:
pdf_solar_merge: pd.DataFrame = pd.merge(solar_capacity_group_by_year, pdf_solar_prices, how = 'inner', on = 'Ano')
pdf_solar_merge

In [ ]:
pdf_solar_merge.plot(x='Capacidade Acumulada [GW]', y='R$/W', grid=True);

### Criando Banco de Dados

In [ ]:
energia_renovavel: DataBaseSQLite3 = DataBaseSQLite3(db_name = 'energia_renovavel3.db')
energia_renovavel.create_database()

### Carregando Banco de Dados

In [ ]:
energia_renovavel.send_dataframe_to_database(
    dataframe = pdf_solar_merge,
    table_name = 'ResumoCapacidadeAnual')

### Verificando Dados

In [ ]:
energia_renovavel.check_tables_in_database()